In [1]:
!pip install kaggle
!pip install tiktoken
!pip install tensorflow


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 33.5 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 29.2 MB/s eta 0:00:0000:01
  Attempting uninstall: keras
    Found existing installation: keras 3.2.1
    Uninstalling keras-3.2.1:
      Successfully uninstalled keras-3.2.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.


In [2]:
!pip install -U nltk==3.8.1.

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 26.1 MB/s eta 0:00:0000:01
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.4
    Uninstalling nltk-3.2.4:
      Successfully uninstalled nltk-3.2.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
preprocessing 0.1.13 requires nltk==3.2.4, but you have nltk 3.8.1 which is incompatible.


In [6]:
#from google.colab import drive
#drive.mount('/content/drive')
import os
os.listdir('/kaggle/input/')

['song_lyrics.csv']

In [3]:
import pandas as pd
import numpy as np
import random
import nltk
import tensorflow as tf
import matplotlib.pyplot as plt
import tiktoken
import os
import math
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import classification_report, accuracy_score, ConfusionMatrixDisplay, confusion_matrix
from sklearn.model_selection import train_test_split, GridSearchCV
from spacy.lang.en.stop_words import STOP_WORDS as en_stop
from collections import Counter, defaultdict
import torch
from torch.nn import functional as F

2024-05-08 14:18:35.979113: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-08 14:18:35.979331: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-08 14:18:36.136943: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


On fait un preprocessing des données

In [7]:
skip_func = lambda x: x % 30 != 0
def split_data(data: pd.DataFrame):
    """
    Split the data in a training and test set.
    The training set is composed of 80% of the data.
    The test set is composed of 20% of the data.
    Useful links: https://towardsdatascience.com/train-validation-and-test-sets-72cb40cba9e7
    """
    train_set = data.sample(frac=0.8, random_state=0)
    validation_set = data.drop(train_set.index)
    return train_set, validation_set
def import_data_n_gram(file_path: str):
    """
    Preprocess the data for the n-gram model.
    The transformation effectuated are:
    - Keep only the english songs and remove the language column
    - Remove the NaN values
    - Replace the square brackets by an empty string to remove the annotations and metadata
    - Keep only the songs with a title length inferior to 50 characters
    - Keep only the songs with a title composed of ASCII characters
    - Concatenate the title and the lyrics in a single column
    - Transform the text in lowercase
    - Split the data in a training and validation set
    Useful links: https://necromuralist.github.io/Neurotic-Networking/posts/nlp/n-gram-pre-processing/index.html
    """
    data = pd.read_csv(file_path, usecols=['title', 'lyrics', 'language_cld3'], skiprows=lambda x: skip_func(x))
    data = data.loc[data['language_cld3'] == 'en']
    data = data.drop(columns=['language_cld3'])
    data = data.dropna()
    data['lyrics'] = data['lyrics'].str.replace(r"\[.*\]", "")
    data = data[data['title'].apply(lambda x: len(x) < 50)]
    data = data[data['title'].str.match(r'^[\x00-\x7F]*$')]
    data['title_lyrics'] = data['title'] + ' ' + data['lyrics']
    data[['title_lyrics', 'title', 'lyrics']] = data[['title_lyrics', 'title', 'lyrics']].apply(lambda x: x.str.lower())
    train_set, validation_set = split_data(data)
    return train_set, validation_set
X_train, X_test = import_data_n_gram('/kaggle/input/song_lyrics.csv')
X_train = X_train['lyrics']
X_test = X_test['lyrics']


On réutilise les tokenizers du cours

In [9]:
#Define tokenizers from course
def lemma_tokenize(doc):
    wnl = WordNetLemmatizer()
    return [wnl.lemmatize(t) for t in word_tokenize(doc)]

def char_tokenize(doc):
    return [char for char in doc]

def byte_tokenize(doc):
    tokens = doc.encode("utf-8")
    tokens = list(map(int, tokens))
    return [str(token) for token in tokens]

def gpt_tokenize(doc):
    enc = tiktoken.encoding_for_model("gpt-4")
    tokens = enc.encode(doc)
    return [str(token) for token in tokens]

Calcul des probabilités du ngram, elles nous serviront à générer les mots.

Calcul des probabilites pour un 5-gram en tokenizant à l'aide de gpt

Calcul de perplexité qui va nous permettre de définir la précision du modèle, plus on augmente le n, ,plus la perplexité du set d'entrainement diminue et celle du set de test augmente

On va générer les mots à l'aide de plusieurs méthode de sampling, d'abord la méthode gloutonne vu en cours, ensuite la méthode top-k et enfin la méthode top-p

In [11]:
def padding(sequence, n, pad_left=False, pad_right=False, left_pad_symbol=None, right_pad_symbol=None):
    if pad_left:
        sequence = [left_pad_symbol] * (n-1) + sequence
    if pad_right:
        sequence += [right_pad_symbol] * (n-1)
    return sequence

In [12]:
nltk.download('all')



[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /usr/share/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /usr/share/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /usr/share/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /usr/share/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /usr/share/nltk_data...
[nltk_data]    |   Package basque_grammars is already up-to-date!
[nltk_data]    | Downloading package bcp47 to /usr/share/nltk_data...
[nltk_data]    | Downloading pa

True

In [14]:
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.lm import MLE, StupidBackoff, Laplace

In [15]:
train_strings = " ".join(X_train)
test_strings = " ".join(X_test)
train_sents = sent_tokenize(train_strings)
print(len(train_sents))
train_sents_tokens = [word_tokenize(s) for s in train_sents]

test_sents = sent_tokenize(test_strings)
print(len(test_sents))
test_sents_tokens = [word_tokenize(s) for s in test_sents]

393092
93239


In [16]:
n = 5

train, vocab = padded_everygram_pipeline(n, train_sents_tokens)



In [17]:
#lm = MLE(n) # Maximum Likelihood Estimate
lm = StupidBackoff(order = n) # Stupid Backoff
#lm = Laplace(n) # Laplace smoothing

In [18]:
lm.fit(train, vocab)
print(lm.vocab)
print(len(lm.vocab))

<Vocabulary with cutoff=1 unk_label='<UNK>' and 249423 items>
249423


In [ ]:
 lm.perplexity(train_sents)

In [20]:
#lm.perplexity(test_sents_tokens) if perplexity not inf

In [21]:
sentence = lm.generate(60, random_seed=42)
print(" ".join(sentence))

na , na , na , na , na ) [ verse 1 : nas ] look at them and who we are is who we blame moments of what you know and you know even though we hold on why does it feel good ? </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
